In [2]:
# coding: utf-8
from __future__ import print_function
from __future__ import division
import tensorflow as tf
from nets import nets_factory
from preprocessing import preprocessing_factory
import reader
import model
import time
import losses
import utils
import os
import argparse

slim = tf.contrib.slim


def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('-c', '--conf', default='conf/beiou.yml', help='配置文件路径')
    return parser.parse_args([])
    

def main(FLAGS):
    style_features_t = losses.get_style_features(FLAGS)
    training_path = os.path.join(FLAGS.model_path, FLAGS.naming)
    if not (os.path.exists(training_path)):
        os.makedirs(training_path)

    with tf.Graph().as_default():
        with tf.Session() as sess:
            """创建Network"""
            network_fn = nets_factory.get_network_fn(
                FLAGS.loss_model,
                num_classes=1,
                is_training=False)

            image_preprocessing_fn, image_unprocessing_fn = preprocessing_factory.get_preprocessing(
                FLAGS.loss_model,
                is_training=False)
 
            """训练图片预处理"""
            processed_images = reader.batch_image(FLAGS.batch_size, FLAGS.image_size, FLAGS.image_size,
                                                  'train2014/', image_preprocessing_fn, epochs=FLAGS.epoch)
            generated = model.transform_network(processed_images, training=True)
            processed_generated = [image_preprocessing_fn(image, FLAGS.image_size, FLAGS.image_size)
                                   for image in tf.unstack(generated, axis=0, num=FLAGS.batch_size)
                                   ]
            processed_generated = tf.stack(processed_generated)
            _, endpoints_dict = network_fn(tf.concat([processed_generated, processed_images], 0), spatial_squeeze=False)
            tf.logging.info('Loss network layers(You can define them in "content_layers" and "style_layers"):')
            for key in endpoints_dict:
                tf.logging.info(key)

            """创建 Losses"""
            content_loss = losses.content_loss(endpoints_dict, FLAGS.content_layers)
            style_loss, style_loss_summary = losses.style_loss(endpoints_dict, style_features_t, FLAGS.style_layers)
            tv_loss = losses.total_variation_loss(generated)  # use the unprocessed image

            loss = FLAGS.style_weight * style_loss + FLAGS.content_weight * content_loss + FLAGS.tv_weight * tv_loss

            """准备训练"""
            global_step = tf.Variable(0, name="global_step", trainable=False)
            variable_to_train = []
            for variable in tf.trainable_variables():
                # 只训练和保存生成网络中的变量
                if not (variable.name.startswith(FLAGS.loss_model)):
                    variable_to_train.append(variable)

            """优化"""
            train_op = tf.train.AdamOptimizer(1e-3).minimize(loss, global_step=global_step, var_list=variable_to_train)

            variables_to_restore = []
            for v in tf.global_variables():
                if not (v.name.startswith(FLAGS.loss_model)):
                    variables_to_restore.append(v)
            saver = tf.train.Saver(variables_to_restore, write_version=tf.train.SaverDef.V1)
            sess.run([tf.global_variables_initializer(), tf.local_variables_initializer()])
            init_func = utils._get_init_fn(FLAGS)
            init_func(sess)
            last_file = tf.train.latest_checkpoint(training_path)
            if last_file:
                tf.logging.info('Restoring model from {}'.format(last_file))
                saver.restore(sess, last_file)

            """开始训练"""
            coord = tf.train.Coordinator()
            threads = tf.train.start_queue_runners(coord=coord)
            start_time = time.time()
            try:
                while not coord.should_stop():
                    _, loss_t, step = sess.run([train_op, loss, global_step])
                    elapsed_time = time.time() - start_time
                    start_time = time.time()
                    if step % 10 == 0:
                        tf.logging.info(
                            'step: %d,  total Loss %f, secs/step: %f,%s' % (step, loss_t, elapsed_time, time.asctime()))
                    """checkpoint"""
                    if step % 50 == 0:
                        tf.logging.info('saving check point...')
                        saver.save(sess, os.path.join(training_path, FLAGS.naming + '.ckpt'), global_step=step)
            except tf.errors.OutOfRangeError:
                saver.save(sess, os.path.join(training_path, 'beiou.ckpt-done'))
                tf.logging.info('Done training -- epoch limit reached')
            finally:
                coord.request_stop()
                tf.logging.info('coordinator stop')
            coord.join(threads)


if __name__ == '__main__':
    tf.logging.set_verbosity(tf.logging.INFO)
    args = parse_args()
    FLAGS = utils.read_conf_file(args.conf)
    main(FLAGS)


INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Use pretrained model vgg_16/vgg_16.ckpt
INFO:tensorflow:Restoring parameters from vgg_16/vgg_16.ckpt
INFO:tensorflow:Target style pattern is saved to: static/img/generated/target_style_beiou.jpg.
INFO:tensorflow:Scale of 0 disables regularizer.
INFO:tensorflow:Loss network layers(You can define them in "content_layers" and "style_layers"):
INFO:tensorflow:vgg_16/conv1/conv1_1
INFO:tensorflow:vgg_16/conv1/conv1_2
INFO:tensorflow:vgg_16/pool1
INFO:tensorflow:vgg_16/conv2/conv2_1
INFO:tensorflow:vgg_16/conv2/conv2_2
INFO:tensorflow:vgg_16/pool2
INFO:tensorflow:vgg_16/conv3/conv3_1
INFO:tensorflow:vgg_16/conv3/conv3_2
INFO:tensorflow:vgg_16/conv3/conv3_3
INFO:tensorflow:vgg_16/pool3
INFO:tensorflow:vgg_16/conv4/conv4_1
INFO:tensorflow:vgg_16/conv4/conv4_2
INFO:tensorflow:vgg_16/conv4/conv4_3
INFO:tensorflow:vgg_16/pool4
INFO:tensorflow:vgg_16/conv5/conv5_1
INFO:tensorflow:vgg_16/conv5/conv5_2
INFO:tensorflow:vgg_16/conv5/conv

INFO:tensorflow:step: 400,  total Loss 205910.890625, secs/step: 13.916219,Tue Jun 11 11:43:59 2019
INFO:tensorflow:saving check point...
INFO:tensorflow:step: 410,  total Loss 227689.906250, secs/step: 14.166193,Tue Jun 11 11:46:20 2019
INFO:tensorflow:step: 420,  total Loss 215042.968750, secs/step: 13.949824,Tue Jun 11 11:48:40 2019
INFO:tensorflow:step: 430,  total Loss 247571.265625, secs/step: 13.925901,Tue Jun 11 11:51:00 2019
INFO:tensorflow:step: 440,  total Loss 341681.281250, secs/step: 13.985369,Tue Jun 11 11:53:22 2019
INFO:tensorflow:step: 450,  total Loss 289719.093750, secs/step: 13.920323,Tue Jun 11 11:55:42 2019
INFO:tensorflow:saving check point...
INFO:tensorflow:Done training -- epoch limit reached
INFO:tensorflow:coordinator stop
